In [1]:
from NER import CRF
import pandas as pd
import time

In [ ]:
%%time
data1 = pd.read_excel('Data/NG.xlsx',sheet_name='Raw')
data = pd.read_excel('Data/data.xlsx',sheet_name='Raw')

In [3]:
data1.columns

Index(['Account', 'Ticket', 'ExternalTicketNumber', 'Description', 'L3', 'L4',
       'Opportunities', 'AutoScripts', 'IPwR', 'Assigned Group',
       'Assigned Name', 'Service Type', 'Service Target', 'Status',
       'Status Reason', 'Priority', 'Service Line', 'Service Sub-Line',
       'Product Name', 'Reported Date & Time', 'Contact Date',
       'Support Respond Date', 'Response Duration - Clock (Seconds)',
       'Resolution Date & Time', 'Resolution Duration - Clock (Seconds)',
       'Submit Date & Time', 'Closed Date & Time', 'Re-Open Date',
       'Age (in Days)', 'OpCat Tier 1', 'OpCat Tier 2', 'OpCat Tier 3',
       'ProdCat Tier 1', 'ProdCat Tier 2', 'ProdCat Tier 3', 'ResCat Tier 1',
       'ResCat Tier 2', 'ResCat Tier 3', 'SLMStatus', 'Resolution', 'Region',
       'Country', 'Site Group', 'City', 'Street', 'Call Source',
       'Resolvable Type', 'Resolution Method', 'Assigned Support Company',
       'Assigned Support Org', 'Submitter Group', 'Submitter', 'Site',
   

In [4]:
data=data[['Description', 'L3', 'L4']]
train=data[(data['L3']!=0)&(data['L4']!=0)]
train.reset_index(drop=True,inplace=True)
test=data[(data['L3']==0)]
test.reset_index(drop=True,inplace=True)

In [5]:
%%time
data1=data1[['Description', 'L3','L4']]
train.columns = ['short_description', 'Products','Issues']
test.columns = ['short_description', 'Products','Issues']
data1.columns = ['short_description', 'Products','Issues']

Wall time: 15 ms


In [6]:
crfobj = CRF.ConditionalRandomField(num_features=2)

In [7]:
# %%time
taggdata=crfobj.train(data1[:200].short_description.astype('str'),[data1.Products.astype('str'),data1.Issues.astype('str')],['P','I'])

In [8]:
X=taggdata

In [9]:
data1.shape

(5091, 3)

In [10]:
def fit(X,y,tag): 
    final_tag=[]
    for sent in X:
        nlp =spacy.load('en_core_web_sm')
        y1 =[nltk.tokenize.regexp_tokenize(i,'[a-zA-Z0-9]+') for i in y[0]]
        flat_list=[item for sublist in y1 for item in sublist]
        tags = []
        for tok,pos,lem,tagg in sent:
            for t in range(len(flat_list)):
                tt=max(len(tok),len(flat_list[t]))
                edit=1-editdistance.eval(tok.lower(),flat_list[t])/tt
                if tok.lower() == flat_list[t].lower() or edit > 0.34:
                    tagg = tag[0]
            tags.append((tok,pos,lem,tagg))
        print(tags)
           # for each document append corresponding tag
        final_tag.append(tags)
    return final_tag

In [ ]:
%%time
import nltk
import editdistance
import spacy
final_tag=fit(X,[data1.Products.astype('str'),data1.Issues.astype('str')],['P','I'])

[('Application', 'NOUN', 'application', 'P'), ('Software', 'PROPN', 'software', 'P'), ('Operating', 'PROPN', 'operating', 'O'), ('System', 'PROPN', 'system', 'O'), ('Issue', 'PROPN', 'issue', 'O'), ('Microsoft', 'PROPN', 'microsoft', 'P'), ('Office', 'PROPN', 'office', 'P'), ('2010', 'NUM', '2010', 'O'), ('CSCGBRAH500408', 'NOUN', 'cscgbrah500408', 'O'), ('T', 'NOUN', 't', 'O')]
[('11', 'NUM', '11', 'O'), ('01', 'NUM', '01', 'P'), ('STK1', 'PROPN', 'stk1', 'O'), ('Urgent', 'PROPN', 'urgent', 'O'), ('FAULT', 'PROPN', 'fault', 'O'), ('LAPTOP', 'VERB', 'laptop', 'P'), ('unable', 'ADJ', 'unable', 'O'), ('to', 'PART', 'to', 'O'), ('access', 'VERB', 'access', 'P'), ('home', 'NOUN', 'home', 'P'), ('wifi', 'NOUN', 'wifi', 'O')]
[('Application', 'NOUN', 'application', 'P'), ('Software', 'PROPN', 'software', 'P'), ('Error', 'PROPN', 'error', 'O'), ('Message', 'PROPN', 'message', 'O'), ('Excel', 'PROPN', 'excel', 'P'), ('2010', 'NUM', '2010', 'O'), ('NGUSANA006864', 'PROPN', 'ngusana006864', 'O')

[('Other', 'ADJ', 'other', 'O'), ('Services', 'PROPN', 'services', 'O'), ('Error', 'PROPN', 'error', 'O'), ('Message', 'PROPN', 'message', 'O'), ('Shared', 'PROPN', 'shared', 'P'), ('Services', 'PROPN', 'services', 'O'), ('Citrix', 'PROPN', 'citrix', 'P'), ('TCS', 'PROPN', 'tcs', 'O'), ('Citrix', 'PROPN', 'citrix', 'P'), ('CSCGBRAE513556D', 'PROPN', 'cscgbrae513556d', 'O')]
[('Application', 'NOUN', 'application', 'P'), ('Software', 'PROPN', 'software', 'P'), ('Error', 'NOUN', 'error', 'O'), ('Message', 'NOUN', 'message', 'O'), ('HyperSnap', 'NOUN', 'hypersnap', 'O'), ('NGUSANA007017', 'VERB', 'ngusana007017', 'O')]
[('Cadent', 'PROPN', 'cadent', 'O'), ('MySAP', 'PROPN', 'mysap', 'P')]
[('11', 'NUM', '11', 'O'), ('01', 'NUM', '01', 'P'), ('DFCC', 'PROPN', 'dfcc', 'O'), ('URGENT', 'PROPN', 'urgent', 'O'), ('Fault', 'PROPN', 'fault', 'O'), ('Outlook', 'PROPN', 'outlook', 'P'), ('asking', 'VERB', 'ask', 'O'), ('for', 'ADP', 'for', 'O'), ('credentials', 'NOUN', 'credential', 'O')]
[('SRX', 

In [ ]:
def fit1(X,y,tag):
    nlp =spacy.load('en_core_web_sm')
    y2 = [nltk.tokenize.regexp_tokenize(i,'[a-zA-Z0-9]+') for i in y[1]]
    flat_list1= [item for sublist in y2 for item in sublist]  
    lemm=[k.lemma_ for i in flat_list1 for k in nlp(i.lower())]
    final_tag1=[]
    for sent in X:
        tags = []
        for tok,pos,lem,tagg in sent:
            for t in range(len(lemm)):
                tt=max(len(lem),len(lemm[t]))
                edit=1-editdistance.eval(lem.lower(),lemm[t])/tt
                if lem.lower() == lemm[t].lower() or edit > 0.34:
                    tagg = 'I'
            tags.append((tok,pos,lem,tagg))
        final_tag1.append(tags)
    return final_tag1

In [ ]:
final_tag1=fit1(final_tag,[data1.Products.astype('str'),data1.Issues.astype('str')],['P','I'])

In [ ]:
final_tag1

In [ ]:
crfobj.train1(final_tag1,[data1.Products.astype('str'),data1.Issues.astype('str')],['P','I'])

In [8]:
%%time
result=crfobj.predict(train.short_description.astype(str))

Wall time: 1min 6s


In [12]:
final= pd.DataFrame(train.short_description)
# final=final.reset_index(drop=True)
# finalresult['Description']=final['short_description']

In [13]:
final['result']=result

In [14]:
final

,short_description,result
0,11/01 STK1 * Urgent* FAULT: LAPTOP- unable to ...,"[O, O, O, O, I, O, O, O, I, O, O]"
1,Application/Software - Error Message - Excel 2...,"[O, O, I, O, P, O, O]"
2,11/01 STK1 Network - Access - UK Shared Areas,"[O, O, O, P, I, O, O, I]"
3,FAULT: Cadent Hub Home page is just a blank page,"[I, O, P, O, O, O, O, O, O, O]"
4,FAULT: .box unable to update and receive mails,"[O, P, O, O, O, O, O, O]"
5,10/31 STK3 Fault: Internet Explorer V8- asking...,"[O, O, O, O, P, P, O, O, O, I, O]"
6,31/10 STK3 FAULT : Printer - Unable to find Pr...,"[O, O, O, I, P, O, O, O, P]"
7,10/31 STK3 JAVA URGENT: Fault: Laptop- user un...,"[O, O, O, P, O, I, P, O, O, O, O, O, O, O, O]"
8,10/31 STK3 JAVA Urgent Fault Laptop - Java upd...,"[O, O, O, P, O, I, P, P, O, O]"
9,10/31 STK3 Fault: Desktop -Desktop is running...,"[O, O, O, I, P, P, O, O, O, O]"


In [25]:
import re
final['description'] = [re.findall('[A-Za-z0-9]+',i) for i in final.short_description]

In [26]:
# mapping each word to corresponding tag
dictionary = [dict(zip(i,j)) for i,j in zip(final.description,result)]

In [27]:
pred_product = []
pred_issue = []
# for index and item in list
for idx,i in enumerate(dictionary):
    for word,tag in i.items():
        if tag == 'P':
            # appending index and data 
            pred_product.append((idx,word.upper()))
        if tag == 'I':
            # appending index and data
            pred_issue.append((idx,word.upper()))


In [29]:
# converting to a data frame
mer = pd.DataFrame(final.short_description)
# adding a index column 
mer['idx'] = mer.index

In [18]:



# Merging products and issues with corresponding data 
p = mer.merge(pd.DataFrame(pred_product,columns=['idx','pro']).groupby('idx').agg(lambda x:set(x.tolist())),on = 'idx',how = 'outer').reset_index()
i = p.merge(pd.DataFrame(pred_issue, columns = ['idx','iss']).groupby('idx').agg(lambda x: set(x.tolist())), on = 'idx', how = 'outer').reset_index()

products = [i.strip('{}').replace("'","").replace('nan','NOT ASSIGNED') for i in i.pro.astype('str')]
issues = [i.strip('{}').replace("'","").replace('nan','NOT ASSIGNED') for i in i.iss.astype('str')]

i['products'] = products
i['issues'] = issues                              



IndexError: list index out of range

In [ ]:
issues=func(final,'I')
finalresult=pd.DataFrame()
finalresult['Products']=products
finalresult['Issues']=issues

In [ ]:
 def fit(self,X,y,tag): 
        self.y=y
        self.tag=tag
        a = len(y)
        b = len(tag)
        if  (a == b == 2) or (a == b == 1):
            # preprocessing target data
            y1 = [nltk.tokenize.regexp_tokenize(i,'[a-zA-Z0-9]+') for i in y[0]]
            # loading spacy 
            nlp = spacy.load('en_core_web_sm')
            # Tagging tag1
            final_tag = []
            # for each document and corrsponding target
            for sent,target in zip(X,y1):
                tags = []
                for tok,pos,lem,tagg in sent:
                    # for each product
                    for t in target:
                        # if target is eual to token
                        tt = max(len(tok),len(t))
                        edit = 1-editdistance.eval(tok.lower(),t)/tt
                        if tok.lower() == t.lower() or edit > 0.34:
                            # tagging it as tag1
                            tagg = tag[0]
                    # appending product tag
                    tags.append((tok,pos,lem,tagg))
                # for each document append corresponding tag
                final_tag.append(tags)
            # Tokenizing
            y2 = [nltk.tokenize.regexp_tokenize(i,'[a-zA-Z0-9]+') for i in y[1]]
            # Lemmatizing
            lem = []
            for i in y2:
                temp = []
                for j in i:
                    for k in nlp(j.lower()):
                        temp.append(k.lemma_)
                lem.append(temp)
            # Tagging tag2
            final_tag2 = []
            # for each document and corrsponding target
            for sent,target in zip(final_tag,lem):
                tags = []
                for tok,pos,lem,tagg in sent:
                    # for each issue
                    for i in target:
                        # if target is eual to lemmatized token 
                        tt = max(len(lem),len(i))
                        edit = 1-editdistance.eval(lem.lower(),i)/tt
                        if (edit > 0.34)|(lem.lower() == i):
                            # tagging it as I
                            tagg = tag[1]
                    # appending tag2
                    tags.append((tok,pos,lem,tagg))
                # for each document append corresponding tag
                final_tag2.append(tags)
                

In [16]:
import nltk
import spacy
nlp = spacy.load('en_core_web_sm')
y1 = [nltk.tokenize.regexp_tokenize(i,'[a-zA-Z0-9]+') for i in data1.Products.astype('str')]
flat_list= [item for sublist in y1 for item in sublist]
        


In [18]:
lem = []
for i in flat_list[:10]:
    for k in nlp(i.lower()):
        lem.append(k.lemma_)

In [19]:
lem

['ms',
 'office',
 'laptop',
 'excel',
 'network',
 'hub',
 'box',
 'account',
 'printer',
 'java']

In [27]:
import editdistance
final_tag = []
for sent in X:
    tags = []
    for tok,pos,lem,tagg in sent:
        for t in range(len(flat_list)):
            tt=max(len(tok),len(flat_list[t]))
            edit=1-editdistance.eval(tok.lower(),flat_list[t])/tt
            if tok.lower() == flat_list[t].lower() or edit > 0.34:
                tagg = 'P'
        tags.append((tok,pos,lem,tagg))
    final_tag.append(tags)

KeyboardInterrupt: 

In [25]:
y2 = [nltk.tokenize.regexp_tokenize(i,'[a-zA-Z0-9]+') for i in data1.Issues.astype('str')]
flat_list1= [item for sublist in y2 for item in sublist]       
final_tag1 = []
for sent in final_tag:
    tags = []
    for tok,pos,lem,tagg in sent:
        for t in range(len(flat_list1)):
            tt=max(len(tok),len(flat_list1[t]))
            edit=1-editdistance.eval(tok.lower(),flat_list1[t])/tt
            if tok.lower() == flat_list1[t].lower() or edit > 0.34:
                tagg = 'I'
        tags.append((tok,pos,lem,tagg))
    final_tag1.append(tags)

In [26]:
final_tag1

[[('Application', 'NOUN', 'application', 'P'),
  ('Software', 'PROPN', 'software', 'P'),
  ('Operating', 'PROPN', 'operating', 'O'),
  ('System', 'PROPN', 'system', 'O'),
  ('Issue', 'PROPN', 'issue', 'I'),
  ('Microsoft', 'PROPN', 'microsoft', 'P'),
  ('Office', 'PROPN', 'office', 'P'),
  ('2010', 'NUM', '2010', 'O'),
  ('CSCGBRAH500408', 'NOUN', 'cscgbrah500408', 'O'),
  ('T', 'NOUN', 't', 'O')],
 [('11', 'NUM', '11', 'O'),
  ('01', 'NUM', '01', 'I'),
  ('STK1', 'PROPN', 'stk1', 'O'),
  ('Urgent', 'PROPN', 'urgent', 'O'),
  ('FAULT', 'PROPN', 'fault', 'I'),
  ('LAPTOP', 'VERB', 'laptop', 'P'),
  ('unable', 'ADJ', 'unable', 'I'),
  ('to', 'PART', 'to', 'I'),
  ('access', 'VERB', 'access', 'I'),
  ('home', 'NOUN', 'home', 'P'),
  ('wifi', 'NOUN', 'wifi', 'O')],
 [('Application', 'NOUN', 'application', 'P'),
  ('Software', 'PROPN', 'software', 'P'),
  ('Error', 'PROPN', 'error', 'I'),
  ('Message', 'PROPN', 'message', 'O'),
  ('Excel', 'PROPN', 'excel', 'P'),
  ('2010', 'NUM', '2010', '